In [ ]:
from lxml import html
import requests
import unicodedata

In [ ]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

In [ ]:
from typing import List
import re

class Game:

    def __init__(self, tree):
        self.nome = self.to_string(tree.xpath("//h1[@id='title']/span/text()"))
        self.prezzo = self.fix_list_string(self.get_prezzo(tree))

        self.piattaforma = self.to_string(tree.xpath("//div[@id='platformInformation_feature_div']/text()[2]"))
        self.classificazione = self.to_string(tree.xpath("//div[@id='vgRating_feature_div']/span[2]/a/text()"))
        
        self.info = self.fix_list_string(tree.xpath("//div[@id='feature-bullets']/ul/li/span/text()"))
        self.data_uscita = self.to_string(tree.xpath('//span[contains(.,"Data d\'uscita")]/span[2]/text()'))
        
        self.n_recensioni = self.to_string(tree.xpath("//div[@id='averageCustomerReviews']/span[3]/a/span/text()"))
        self.val_recensioni = self.to_string(tree.xpath("//div[@id='averageCustomerReviews']/span/span/@title"))
    
    def print_prodotto(self):
        print('Nome: ' + self.nome)
        print('Prezzo: ' + self.to_string(self.prezzo))
        print('Piattaforma: ' + self.piattaforma)
        print('Classificazione: ' + self.classificazione)
        print('Info articolo: ' + self.to_string(self.info))
        print('Data di uscita: ' + self.data_uscita)
        print('Numero recensioni: ' + self.n_recensioni)
        print('Valutazione: ' + self.val_recensioni + '\n')

    def fix_list_string(self, stringa: List[str]):
        lista = []
        for s in stringa:
            if len(stringa) > 1:
                lista.append(unicodedata.normalize("NFKD", s.strip()))
            else:
                return unicodedata.normalize("NFKD", s.strip())
        return lista

    def to_string(self, stringa: List[str]):
        str1 = ""
        for s in stringa:
            if len(stringa) > 1:
                str1 += '\n- '
            str1 += unicodedata.normalize("NFKD", s.strip())
        return str1

    def get_prezzo(self, tree):
        lista_prezzi = []
        prezzi = tree.xpath("//div[@id='corePriceDisplay_desktop_feature_div']/div[1]/span/span/text()")
        if (len(prezzi) > 1):
            lista_prezzi.append("Nuovo: " + prezzi[0])
            lista_prezzi.append("Usato: " + prezzi[1])
        elif (len(prezzi) == 1):
            lista_prezzi.append("Nuovo: " + prezzi[0])
        return lista_prezzi

In [ ]:
links = [
        "https://amzn.eu/d/iV7kobw",
        "https://amzn.eu/d/71sSdyJ",
        "https://amzn.eu/d/0qN2svy",
        "https://amzn.eu/d/aNpuTyK",
        "https://amzn.eu/d/hSA3iTJ",
        "https://amzn.eu/d/9RbDR0Z",
        "https://amzn.eu/d/csA6b0b",
        "https://amzn.eu/d/7KHyX18",
        "https://amzn.eu/d/iCRnEbr",
        "https://amzn.eu/d/fRlirY8"
        ]

In [ ]:
games:List[Game] = []
for link in links:
    page = requests.get(link, header)
    if page.status_code != 503:
        game = Game(html.fromstring(page.content))
        games.append(game)
        # game.print_prodotto()
    else: 
        page = requests.get(link, header)
        if page.status_code != 503:
            game = Game(html.fromstring(page.content))
            games.append(game)
            # game.print_prodotto()
        else:
            print("503 Error code")

In [ ]:
import pandas as pd

lst = [[x.nome, x.prezzo, x.piattaforma, x.classificazione, x.info, x.data_uscita, x.n_recensioni, x.val_recensioni] for x in games]
df = pd.DataFrame(lst)
df.to_csv('games.csv', index=0, header=['Nome', 'Prezzo', 'Piattaforma', 'Classificazione', 'Info', 'Data di uscita', 'Numero recensioni', 'Valutazione'])